In [1]:
!pip install pandas xlrd ipdb

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import os
import pandas as pd
import ipdb
from datetime import datetime, time

In [3]:
%ls data

'weekly-2020-02-10 0925.xlsx'


In [4]:
libs = sorted(pd.read_csv('libraries.csv').full_name.tolist())

In [5]:
# CONVERT
dn = 'data'
fn = dn + '/' + 'weekly-2020-02-10 0925.xlsx'
xl = pd.ExcelFile(fn)
sheets = xl.sheet_names
res=[]
for sheet in sheets:
    df = xl.parse(sheet)
    date = datetime.strptime(df.columns[0], '%d %B %Y')
    df = (
        df.loc[df['Counter Name'].isin(libs)]
        .rename(columns={df.columns[0]: 'dt'})
        .fillna('x')        
    )
    times=df.loc[df.dt!='x']
    df.dt = df.dt.replace(to_replace='x', method='ffill')
    dt = list(map(lambda x: datetime.combine(date, time(int(x[:2]), 0)), df.dt.tolist()))
    df.dt = dt
    res.append(df)                             

df = pd.concat(res)
df = df.rename(columns={'Counter Name': 'Library'})

In [6]:
oh = pd.read_csv('opening hours.csv')
oh

,library,day,start,finish,opening_type
0,Burnt Oak,Mon,14:00:00,20:00:00,staffed
1,Burnt Oak,Wed,09:00:00,12:30:00,staffed
2,Burnt Oak,Fri,14:00:00,17:00:00,staffed
3,Burnt Oak,Sat,14:00:00,17:00:00,staffed
4,Burnt Oak,Mon,09:00:00,14:00:00,SSO
...,...,...,...,...,...
144,North Finchley,Thu,12:30:00,20:00:00,SSO
145,North Finchley,Fri,17:00:00,20:00:00,SSO
146,East Finchley,Thu,17:00:00,20:00:00,SSO
147,Osidge,Thu,17:00:00,20:00:00,SSO


In [13]:
check = df.iloc[0][['dt', 'Library']]

def get_ot(dt, lib):
    ipdb.set_trace()
    day = dt.day_name()[:3]
    mask = (
        (oh.library==lib) &
        (oh.day==day) &
        (int(oh.start[:2])<=dt.hour) &
        (int(oh.finish[:2])>dt.hour)
    )
    ot = oh.loc[mask]
    ipdb.set_trace()
    return None

get_ot(check['dt'], check.Library)

> <ipython-input-13-31e1ee62d56e>(5)get_ot()
      4     ipdb.set_trace()
----> 5     day = dt.day_name()[:3]
      6     mask = (

ipdb> c


TypeError: cannot convert the series to <class 'int'>